Requirements.txt

torch 2.0.0+cu117
transformers 4.28.1
numpy
typing
tqdm
gc
pandas
collections
sklearn
contextlib
io
IPython
os

In [1]:
!git clone https://<your_token>@github.com/JuliaKudryavtseva/NLP-DL-Project-hypo-to-hyper.git


Cloning into 'NLP-DL-Project-hypo-to-hyper'...
remote: Enumerating objects: 111, done.
remote: Counting objects: 100% (111/111), done.
remote: Compressing objects: 100% (86/86), done.
remote: Total 111 (delta 21), reused 106 (delta 20), pack-reused 0
Receiving objects: 100% (111/111), 4.79 MiB | 7.70 MiB/s, done.
Resolving deltas: 100% (21/21), done.


In [2]:
!pip install torch transformers -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 77.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 58.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.2/224.2 kB 11.7 MB/s eta 0:00:00


In [3]:
!nvidia-smi

Mon Apr 24 13:19:51 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P8    12W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
import os
import sys

In [2]:
#os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [3]:
import torch

if torch.cuda.is_available():
    device='cuda'
    print('GPU')
else:
    device='cpu'
    print('CPU')
    
    
SEED = 0
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)

GPU


In [19]:
import pandas as pd

path_data_en = "/content/NLP-DL-Project-hypo-to-hyper/SemEval2018-Task9/training/data/1A.english.training.data.txt"
path_gold_en = "/content/NLP-DL-Project-hypo-to-hyper/SemEval2018-Task9/training/gold/1A.english.training.gold.txt"

train_data_en_data = pd.read_csv(path_data_en, header=None, sep="\t", names=['term', 'relation'])
train_gold_en_data = pd.read_csv(path_gold_en, header=None, names=['hypernym'])

train_data_en_data.head()

path_test_data_en = "/content/NLP-DL-Project-hypo-to-hyper/SemEval2018-Task9/test/data/1A.english.test.data.txt"
path_test_gold_en = "/content/NLP-DL-Project-hypo-to-hyper/SemEval2018-Task9/test/gold/1A.english.test.gold.txt"

test_data_en_data = pd.read_csv(path_test_data_en, header=None, sep="\t", names=['term', 'relation'])
test_gold_en_data = pd.read_csv(path_test_gold_en, header=None, names=['hypernym'])

def standard_preprcessing(train_features, train_target, test_features, test_target):
    
    train_data_en = train_features.copy()
    train_data_en = 'hyperonym: ' + train_data_en.term + ' | hyponyms:'
    print(train_data_en.head())

    train_gold_en = train_target.copy()
    train_gold_en = train_gold_en.hypernym.str.split('\t').str.join(', ')
    print(train_gold_en.head())
    
    test_data_en = test_features.copy()
    test_data_en = 'hyperonym: ' + test_data_en.term + ' | hyponyms:'
    print(test_data_en.head())

    test_gold_en = test_target.copy()
    test_gold_en = test_gold_en.hypernym.str.split('\t').str.join(', ')
    print(test_gold_en.head())
    
    return train_data_en, train_gold_en, test_data_en, test_gold_en


train_data_en, train_gold_en, test_data_en, test_gold_en = standard_preprcessing(train_data_en_data, 
                                                                                 train_gold_en_data, 
                                                                                 test_data_en_data, 
                                                                                 test_gold_en_data)

0          hyperonym: blackfly | hyponyms:
1          hyperonym: Turonian | hyponyms:
2        hyperonym: abhorrence | hyponyms:
3    hyperonym: tropical storm | hyponyms:
4    hyperonym: militarization | hyponyms:
Name: term, dtype: object
0                           homopterous insect, insect
1    technical specification, geologic timescale, p...
2                      distaste, hatred, hate, disgust
3    atmosphere, windstorm, violent storm, air curr...
4                                       social control
Name: hypernym, dtype: object
0    hyperonym: maliciousness | hyponyms:
1          hyperonym: buckler | hyponyms:
2        hyperonym: spelunker | hyponyms:
3     hyperonym: quo warranto | hyponyms:
4     hyperonym: Jeff Francis | hyponyms:
Name: term, dtype: object
0       malevolence, distaste, hatred, hate, malignity
1                                           body armor
2                    exploration, adventurer, explorer
3    proceedings, legal proceedings, proceeding, du..

# Experiment

In [20]:
from transformers import T5ForConditionalGeneration, AutoTokenizer
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, AutoConfig

In [21]:
model_checkpoint = "t5-small"
model = T5ForConditionalGeneration.from_pretrained(model_checkpoint).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, max_length=100, block_size=64)

In [22]:
#sys.path.append("/content/NLP-DL-Project-hypo-to-hyper/")
os.chdir('/content/NLP-DL-Project-hypo-to-hyper')

In [23]:
from evaluation.data_collat import DataCollatorWithPadding
from evaluation.experiment import Experiment

1. train params/predict params [1, 1]
2. train / params predict [0, 1]
3. params predict [0, 0]

## Change only parmeters in generating with no training

In [24]:
check_param = {'GenArgs': {'num_beams': [2, 10]}}

freezed_param = {'TrainArgs': {'num_train_epochs':1, 'per_device_train_batch_size':16, 'save_steps':1}, 
                 'GenArgs': {'max_length': 3, 'early_stopping': True}, 
                 'SelectStrategy': None, 'PredForm': None}


In [25]:
!mkdir output

mkdir: cannot create directory ‘output’: File exists


In [29]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


path_data = '/content/NLP-DL-Project-hypo-to-hyper/SemEval2018-Task9/test/gold/1A.english.test.gold.txt'

experiment1 = Experiment(output_dir='output/',
                         check_param=check_param, freezed_param=freezed_param,
                         model=model, tokenizer=tokenizer, device=device,
                         data_collator=data_collator,
                         data_train=train_data_en.tolist(), target_train=train_gold_en.tolist(), 
                         data_test=test_data_en.tolist(), target_test=test_gold_en.tolist(), strategy=[0, 0], 
                         path_to_test=path_data
                        )

In [31]:
experiment1.run_experiment()

100%|██████████| 1500/1500 [00:45<00:00, 32.90it/s]


,MRR,MAP,P@1,P@3,P@5,P@15
Meta-Params,,,,,,
{'GenArgs': {'num_beams': 2}},0.00133,0.00034,0.00133,0.00044,0.00027,0.0002
{'GenArgs': {'num_beams': 10}},0.00133,0.00034,0.00133,0.00044,0.00027,0.0002


## Change only parmeters in generating with training

In [32]:
check_param = {'GenArgs': {'num_beams': [2, 10]}}

freezed_param = {'TrainArgs': {'num_train_epochs':1, 'per_device_train_batch_size':16, 'save_steps':1}, 
                 'GenArgs': {'max_length': 3, 'early_stopping': True}, 
                 'SelectStrategy': None, 'PredForm': None}

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


path_data = '/content/NLP-DL-Project-hypo-to-hyper/SemEval2018-Task9/test/gold/1A.english.test.gold.txt'

experiment1 = Experiment(output_dir='output/',
                         check_param=check_param, freezed_param=freezed_param,
                         model=model, tokenizer=tokenizer, device=device,
                         data_collator=data_collator,
                         data_train=train_data_en.tolist(), target_train=train_gold_en.tolist(), 
                         data_test=test_data_en.tolist(), target_test=test_gold_en.tolist(),
                         strategy=[0, 1], 
                         path_to_test=path_data
                        )

In [33]:
experiment1.run_experiment()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss


100%|██████████| 1500/1500 [00:51<00:00, 29.40it/s]


,MRR,MAP,P@1,P@3,P@5,P@15
Meta-Params,,,,,,
{'GenArgs': {'num_beams': 2}},0.0,0.0,0.0,0.0,0.0,0.0
{'GenArgs': {'num_beams': 10}},0.0,0.0,0.0,0.0,0.0,0.0


## Change generating and training parmeters

In [ ]:
check_param = {'TrainArgs': {'num_train_epochs':[1, 2]}, 'GenArgs': {'num_beams': [2, 10]}}

freezed_param = {'TrainArgs': {'per_device_train_batch_size':16, 'save_steps':1}, 
                 'GenArgs': {'max_length': 3, 'early_stopping': True}, 
                 'SelectStrategy': None, 'PredForm': None}

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


path_data = '/content/NLP-DL-Project-hypo-to-hyper/SemEval2018-Task9/test/gold/1A.english.test.gold.txt'

experiment1 = Experiment(output_dir='/home/jovyan/work',
                         check_param=check_param, freezed_param=freezed_param,
                         model=model, tokenizer=tokenizer, device=device,
                         data_collator=data_collator,
                         data_train=train_data_en.tolist(), target_train=train_gold_en.tolist(), 
                         data_test=test_data_en.tolist(), target_test=test_gold_en.tolist(),
                         strategy=[1, 1], 
                         path_to_test=path_data
                        )

In [ ]:
experiment1.run_experiment()

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss


100%|███████████████████████████████████████████████████████████████████████████████| 1500/1500 [00:37<00:00, 40.19it/s]
/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss


100%|███████████████████████████████████████████████████████████████████████████████| 1500/1500 [00:41<00:00, 36.44it/s]
/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss


100%|███████████████████████████████████████████████████████████████████████████████| 1500/1500 [00:37<00:00, 40.52it/s]
/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss


100%|███████████████████████████████████████████████████████████████████████████████| 1500/1500 [00:41<00:00, 36.30it/s]


,MRR,MAP,P@1,P@3,P@5,P@15
Meta-Params,,,,,,
"{'TrainArgs': {'num_train_epochs': 1}, 'GenArgs': {'num_beams': 2}}",0.00133,0.00052,0.00133,0.00044,0.00044,0.00044
"{'TrainArgs': {'num_train_epochs': 1}, 'GenArgs': {'num_beams': 10}}",0.00467,0.00147,0.00467,0.00156,0.00118,0.00109
"{'TrainArgs': {'num_train_epochs': 2}, 'GenArgs': {'num_beams': 2}}",0.00533,0.00168,0.00533,0.00178,0.00137,0.00124
"{'TrainArgs': {'num_train_epochs': 2}, 'GenArgs': {'num_beams': 10}}",0.008,0.00269,0.008,0.00278,0.00219,0.00209


In [ ]:
!nvidia-smi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Mon Apr 24 00:13:24 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.108.03   Driver Version: 510.108.03   CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:1B:00.0 Off |                  Off |
|  0

In [ ]:
!rm -rf experiment

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
